In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv("../data/cleaned/hcmhouseprice.csv")

df_processed = df.drop('address', axis=1)
df_processed = df_processed.dropna()

feature_cols = ['area', 'bedrooms', 'bathrooms']
target_col = 'price'

X_df = df_processed[feature_cols]
y_series = df_processed[target_col]

X_train_df, X_test_df, y_train_series, y_test_series = train_test_split(
    X_df, y_series, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)

y_train_log = np.log1p(y_train_series)

X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_log.values.reshape(-1, 1), dtype=torch.float32)

X_test = torch.tensor(X_test_scaled, dtype=torch.float32)

print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

Kích thước X_train: torch.Size([1924, 3])
Kích thước y_train: torch.Size([1924, 1])


In [10]:
model = torch.nn.Linear(in_features=X_train.shape[1], out_features=1)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10000
for epoch in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 1000== 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

Epoch 1000/10000, Loss: 0.945549
Epoch 2000/10000, Loss: 0.520333
Epoch 3000/10000, Loss: 0.483809
Epoch 4000/10000, Loss: 0.483295
Epoch 5000/10000, Loss: 0.483295
Epoch 6000/10000, Loss: 0.483295
Epoch 7000/10000, Loss: 0.483295
Epoch 8000/10000, Loss: 0.483295
Epoch 9000/10000, Loss: 0.483295
Epoch 10000/10000, Loss: 0.483295


In [11]:
model.eval() 

with torch.no_grad():
    y_pred_log_tensor = model(X_test)
    y_pred_log = y_pred_log_tensor.numpy().flatten()
    
    y_pred = np.expm1(y_pred_log)
    
    y_true = y_test_series.values.flatten()
    
    y_pred = np.nan_to_num(y_pred, nan=0.0, posinf=y_true.max(), neginf=0.0)
    
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f} (tỷ VNĐ)")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f} (tỷ VNĐ)")

R-squared (R²): -2.6240
Mean Absolute Error (MAE): 6.8435 (tỷ VNĐ)
Root Mean Squared Error (RMSE): 25.3660 (tỷ VNĐ)
